<a href="https://colab.research.google.com/github/booorayan/Regression_fifa/blob/master/Football_Match_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Football Match Outcomes Using Regression Models 

## 1. Defining the Question

### a) Specifying the Question

### b) Defining the Metric for Success

### c) Understanding the context 

### d) Recording the Experimental Design

### e) Data Relevance

## 2. Importing Required Libraries

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

## 3. Reading the Data

In [302]:
# Loading the Dataset
# 
#
ranking = pd.read_csv('fifa_ranking.csv')
ranking.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


In [303]:
ranking.tail()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57792,206,Tonga,TGA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OFC,2018-06-07


In [304]:
results = pd.read_csv('results.csv')
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [305]:
results.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True
40838,2019-07-20,Papua New Guinea,Fiji,1,1,Pacific Games,Apia,Samoa,True


In [0]:
ran_copy = ranking.copy()
res_copy = results.copy()

## 4. Exploring/Checking the Data

In [307]:
# Determining the no. of records in our dataset
#
ranking.shape

(57793, 16)

In [308]:
results.shape

(40839, 9)

In [309]:
# Previewing the top of our dataset
#

ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
rank                       57793 non-null int64
country_full               57793 non-null object
country_abrv               57793 non-null object
total_points               57793 non-null float64
previous_points            57793 non-null int64
rank_change                57793 non-null int64
cur_year_avg               57793 non-null float64
cur_year_avg_weighted      57793 non-null float64
last_year_avg              57793 non-null float64
last_year_avg_weighted     57793 non-null float64
two_year_ago_avg           57793 non-null float64
two_year_ago_weighted      57793 non-null float64
three_year_ago_avg         57793 non-null float64
three_year_ago_weighted    57793 non-null float64
confederation              57793 non-null object
rank_date                  57793 non-null object
dtypes: float64(9), int64(3), object(4)
memory usage: 7.1+ MB


In [310]:
# Previewing the bottom of our dataset
# 
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40839 entries, 0 to 40838
Data columns (total 9 columns):
date          40839 non-null object
home_team     40839 non-null object
away_team     40839 non-null object
home_score    40839 non-null int64
away_score    40839 non-null int64
tournament    40839 non-null object
city          40839 non-null object
country       40839 non-null object
neutral       40839 non-null bool
dtypes: bool(1), int64(2), object(6)
memory usage: 2.5+ MB


In [0]:
# Checking whether each column has an appropriate datatype
#

## 5. Tidying the Dataset

In [312]:
ranking.columns

Index(['rank', 'country_full', 'country_abrv', 'total_points',
       'previous_points', 'rank_change', 'cur_year_avg',
       'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
       'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg',
       'three_year_ago_weighted', 'confederation', 'rank_date'],
      dtype='object')

In [313]:
ranking = ranking.drop(ranking.iloc[:,2:15], 1)
ranking.head()

,rank,country_full,rank_date
0,1,Germany,1993-08-08
1,2,Italy,1993-08-08
2,3,Switzerland,1993-08-08
3,4,Sweden,1993-08-08
4,5,Argentina,1993-08-08


In [314]:
ranking.tail()

,rank,country_full,rank_date
57788,206,Anguilla,2018-06-07
57789,206,Bahamas,2018-06-07
57790,206,Eritrea,2018-06-07
57791,206,Somalia,2018-06-07
57792,206,Tonga,2018-06-07


In [315]:
results.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral'],
      dtype='object')

In [316]:
results = results.drop('city', 1)
results.head()

,date,home_team,away_team,home_score,away_score,tournament,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Scotland,False


In [317]:
res_fil = results[results.neutral == False]
res_fil.tail()

,date,home_team,away_team,home_score,away_score,tournament,country,neutral
40803,2019-07-08,Samoa,Papua New Guinea,0,6,Pacific Games,Samoa,False
40817,2019-07-12,Samoa,Tonga,2,0,Pacific Games,Samoa,False
40821,2019-07-13,India,North Korea,2,5,Intercontinental Cup,India,False
40829,2019-07-16,India,Syria,1,1,Intercontinental Cup,India,False
40832,2019-07-18,Samoa,Vanuatu,0,11,Pacific Games,Samoa,False


In [318]:
res_fil.tournament.unique()

array(['Friendly', 'British Championship', 'Copa Lipton', 'Copa Newton',
       'Copa Premio Honor Argentino', 'Copa Premio Honor Uruguayo',
       'Copa Roca', 'Copa América', 'Copa Chevallier Boutell',
       'Nordic Championship', 'International Cup', 'Baltic Cup',
       'Balkan Cup', 'FIFA World Cup', 'Copa Rio Branco',
       'FIFA World Cup qualification', 'CCCF Championship',
       'NAFU Championship', 'Copa Oswaldo Cruz',
       'Pan American Championship', 'Copa del Pacífico',
       "Copa Bernardo O'Higgins", 'AFC Asian Cup qualification',
       'Atlantic Cup', 'AFC Asian Cup', 'African Cup of Nations',
       'Copa Paz del Chaco', 'UEFA Euro qualification', 'UEFA Euro',
       'Merdeka Tournament', 'Windward Islands Tournament',
       'African Cup of Nations qualification', 'Vietnam Independence Cup',
       'Copa Carlos Dittborn', 'Copa Juan Pinto Durán', 'UAFA Cup',
       'CONCACAF Championship', 'South Pacific Games',
       'CONCACAF Championship qualification', 'Co

In [319]:
# filtering by tournament 
tourna = ['Friendly', 'FIFA World Cup', 'Copa América', 'UEFA Euro']
res_f = res_fil[res_fil['tournament'].isin(tourna)]
res_f.head()


,date,home_team,away_team,home_score,away_score,tournament,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Scotland,False


In [320]:
res_f.tail()

,date,home_team,away_team,home_score,away_score,tournament,country,neutral
40682,2019-06-14,Brazil,Bolivia,3,0,Copa América,Brazil,False
40707,2019-06-18,Brazil,Venezuela,0,0,Copa América,Brazil,False
40761,2019-06-27,Brazil,Paraguay,0,0,Copa América,Brazil,False
40787,2019-07-02,Brazil,Argentina,2,0,Copa América,Brazil,False
40798,2019-07-07,Brazil,Peru,3,1,Copa América,Brazil,False


In [321]:

res_f['year'] = pd.DatetimeIndex(res_f['date']).year
res_f['month'] = pd.DatetimeIndex(res_f['date']).month
res_f['day'] = pd.DatetimeIndex(res_f['date']).day

res_f.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,date,home_team,away_team,home_score,away_score,tournament,country,neutral,year,month,day
0,1872-11-30,Scotland,England,0,0,Friendly,Scotland,False,1872,11,30
1,1873-03-08,England,Scotland,4,2,Friendly,England,False,1873,3,8
2,1874-03-07,Scotland,England,2,1,Friendly,Scotland,False,1874,3,7
3,1875-03-06,England,Scotland,2,2,Friendly,England,False,1875,3,6
4,1876-03-04,Scotland,England,3,0,Friendly,Scotland,False,1876,3,4


In [322]:
r_f = res_f[res_f['year'].between(2006,2018, inclusive=True)]
r_f.head()

,date,home_team,away_team,home_score,away_score,tournament,country,neutral,year,month,day
28033,2006-01-02,Qatar,Libya,2,0,Friendly,Qatar,False,2006,1,2
28034,2006-01-05,Egypt,Zimbabwe,2,0,Friendly,Egypt,False,2006,1,5
28036,2006-01-09,Morocco,DR Congo,3,0,Friendly,Morocco,False,2006,1,9
28038,2006-01-12,Tunisia,Libya,1,0,Friendly,Tunisia,False,2006,1,12
28039,2006-01-14,Egypt,South Africa,1,2,Friendly,Egypt,False,2006,1,14


In [323]:
r_f.dtypes

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
country       object
neutral         bool
year           int64
month          int64
day            int64
dtype: object

In [324]:
r_f.home_score.iloc[0]

2

In [325]:
status = []

for i in range(len(r_f['home_team'])):
  if r_f.home_score.iloc[i] > r_f.away_score.iloc[i]:
    status.append('Win')
  elif r_f['home_score'].iloc[i] < r_f.away_score.iloc[i]:
    status.append('Lose')
  else:
    status.append('Draw')
  
  
    
    
#     if (r_f[''][i] > j):
#       status.append('Win')
#     elif (i < j):
#       status.append('Lose')
#     else:
#       status.append('Draw')
      
# #   if (r_f['home_score'][i]) > (r_f['away_score'][i]):
# #     status.append('Win')
# #   elif r_f['home_score'][i] < r_f['away_score'][i]:
# #     status.append('Lose')
# #   else:
# #     status.append('Draw')

    
r_f['status'] = status
# r_f.apply(lambda col: col.drop_duplicates().reset_index(drop=True))

r_f.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,date,home_team,away_team,home_score,away_score,tournament,country,neutral,year,month,day,status
28033,2006-01-02,Qatar,Libya,2,0,Friendly,Qatar,False,2006,1,2,Win
28034,2006-01-05,Egypt,Zimbabwe,2,0,Friendly,Egypt,False,2006,1,5,Win
28036,2006-01-09,Morocco,DR Congo,3,0,Friendly,Morocco,False,2006,1,9,Win
28038,2006-01-12,Tunisia,Libya,1,0,Friendly,Tunisia,False,2006,1,12,Win
28039,2006-01-14,Egypt,South Africa,1,2,Friendly,Egypt,False,2006,1,14,Lose
28040,2006-01-14,Morocco,Zimbabwe,1,0,Friendly,Morocco,False,2006,1,14,Win
28041,2006-01-14,Senegal,DR Congo,0,0,Friendly,Senegal,False,2006,1,14,Draw
28042,2006-01-15,Tunisia,Ghana,2,0,Friendly,Tunisia,False,2006,1,15,Win
28044,2006-01-17,Morocco,Angola,2,2,Friendly,Morocco,False,2006,1,17,Draw
28045,2006-01-18,Saudi Arabia,Sweden,1,1,Friendly,Saudi Arabia,False,2006,1,18,Draw


In [326]:
res_f = res_f[res_f['year'] == 2018]
res_f.head()

fil_res = res_f[~(res_f['month'] > 6)]
fil_res.head()

,date,home_team,away_team,home_score,away_score,tournament,country,neutral,year,month,day
39501,2018-01-11,Indonesia,Iceland,0,6,Friendly,Indonesia,False,2018,1,11
39504,2018-01-14,Indonesia,Iceland,1,4,Friendly,Indonesia,False,2018,1,14
39531,2018-01-28,United States,Bosnia and Herzegovina,0,0,Friendly,United States,False,2018,1,28
39534,2018-01-28,Yorkshire,Ellan Vannin,1,1,Friendly,England,False,2018,1,28
39543,2018-02-26,Saudi Arabia,Moldova,3,0,Friendly,Saudi Arabia,False,2018,2,26


In [327]:
res_f.head()

,date,home_team,away_team,home_score,away_score,tournament,country,neutral,year,month,day
39501,2018-01-11,Indonesia,Iceland,0,6,Friendly,Indonesia,False,2018,1,11
39504,2018-01-14,Indonesia,Iceland,1,4,Friendly,Indonesia,False,2018,1,14
39531,2018-01-28,United States,Bosnia and Herzegovina,0,0,Friendly,United States,False,2018,1,28
39534,2018-01-28,Yorkshire,Ellan Vannin,1,1,Friendly,England,False,2018,1,28
39543,2018-02-26,Saudi Arabia,Moldova,3,0,Friendly,Saudi Arabia,False,2018,2,26


In [328]:
res_fi = fil_res.drop(['date','neutral'], 1)
res_fi.tail()

,home_team,away_team,home_score,away_score,tournament,country,year,month,day
39859,Belgium,Costa Rica,4,1,Friendly,Belgium,2018,6,11
39861,Poland,Lithuania,4,0,Friendly,Poland,2018,6,12
39862,Russia,Saudi Arabia,5,0,FIFA World Cup,Russia,2018,6,14
39878,Russia,Egypt,3,1,FIFA World Cup,Russia,2018,6,19
39894,Russia,Uruguay,0,3,FIFA World Cup,Russia,2018,6,25


In [329]:
res_fi[(res_fi['day'] > 7) & (res_fi['month'] == 6)]

,home_team,away_team,home_score,away_score,tournament,country,year,month,day
39834,Germany,Saudi Arabia,2,1,Friendly,Germany,2018,6,8
39835,Croatia,Senegal,2,1,Friendly,Croatia,2018,6,8
39836,Switzerland,Japan,2,0,Friendly,Switzerland,2018,6,8
39838,Poland,Chile,2,2,Friendly,Poland,2018,6,8
39841,Denmark,Mexico,2,0,Friendly,Denmark,2018,6,9
39842,Sweden,Peru,0,0,Friendly,Sweden,2018,6,9
39843,Hungary,Australia,1,2,Friendly,Hungary,2018,6,9
39844,Estonia,Morocco,1,3,Friendly,Estonia,2018,6,9
39845,Latvia,Azerbaijan,1,3,Friendly,Latvia,2018,6,9
39846,Finland,Belarus,2,0,Friendly,Finland,2018,6,9


In [330]:
ranking['year'] = pd.DatetimeIndex(ranking['rank_date']).year
ranking['month'] = pd.DatetimeIndex(ranking['rank_date']).month
ranking['day'] = pd.DatetimeIndex(ranking['rank_date']).day

ranking.head()

,rank,country_full,rank_date,year,month,day
0,1,Germany,1993-08-08,1993,8,8
1,2,Italy,1993-08-08,1993,8,8
2,3,Switzerland,1993-08-08,1993,8,8
3,4,Sweden,1993-08-08,1993,8,8
4,5,Argentina,1993-08-08,1993,8,8


In [331]:
rank_f = ranking[ranking['year'].between(2006,2018,inclusive=True)]
rank_f.head()

,rank,country_full,rank_date,year,month,day
26938,1,Brazil,2006-01-18,2006,1,18
26939,2,Czech Republic,2006-01-18,2006,1,18
26940,3,Netherlands,2006-01-18,2006,1,18
26941,4,Argentina,2006-01-18,2006,1,18
26942,5,Spain,2006-01-18,2006,1,18


In [332]:
rank_f.tail()

,rank,country_full,rank_date,year,month,day
57788,206,Anguilla,2018-06-07,2018,6,7
57789,206,Bahamas,2018-06-07,2018,6,7
57790,206,Eritrea,2018-06-07,2018,6,7
57791,206,Somalia,2018-06-07,2018,6,7
57792,206,Tonga,2018-06-07,2018,6,7


In [333]:
ran_f = rank_f.drop('rank_date', 1)
ran_f.tail()

,rank,country_full,year,month,day
57788,206,Anguilla,2018,6,7
57789,206,Bahamas,2018,6,7
57790,206,Eritrea,2018,6,7
57791,206,Somalia,2018,6,7
57792,206,Tonga,2018,6,7


In [211]:
gr = ran_f.groupby(['country_full', 'rank'])['month'].unique()
gr

# rank changes almost every month

country_full         rank
Afghanistan          140              [5]
                     142              [4]
                     145              [6]
                     148        [1, 2, 3]
Albania              56            [4, 5]
                     58            [3, 6]
                     59               [2]
                     60               [1]
Algeria              57               [1]
                     60            [2, 3]
                     62               [4]
                     64               [5]
                     66               [6]
American Samoa       191           [4, 5]
                     192              [6]
                     194        [1, 2, 3]
Andorra              130           [3, 6]
                     132           [4, 5]
                     136              [1]
                     137              [2]
Angola               137              [6]
                     138           [4, 5]
                     141           [2, 3]
        

In [212]:
ran_f[ran_f['month'] == 5].head()

,rank,country_full,year,month,day
57371,1,Germany,2018,5,17
57372,2,Brazil,2018,5,17
57373,3,Belgium,2018,5,17
57374,4,Portugal,2018,5,17
57375,5,Argentina,2018,5,17


In [0]:
# Checking for Anomalies
#

In [0]:
# Identifying the Missing Data
#

In [0]:
# Dealing with the Missing Data
#

In [0]:
# More data cleaning procedures
#

## 6. Exploratory Analysis

In [0]:
# Ploting the univariate summaries and recording our observations
#

## 7. Implementing the Solution

In [0]:
# Implementing the Solution
# 

## 8. Challenging the solution

> The easy solution is nice because it is, well, easy, but you should never allow those results to hold the day. You should always be thinking of ways to challenge the results, especially if those results comport with your prior expectation.






In [0]:
# Reviewing the Solution 
#

## 9. Follow up questions

> At this point, we can refine our question or collect new data, all in an iterative process to get at the truth.



### a). Did we have the right data?

### b). Do we need other data to answer our question?

### c). Did we have the right question?